In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import transformers as ppb
import pandas as pd

# Read the CSV into a pandas DataFrame
df = pd.read_csv('../output/acoustic/transcriptions.csv')

# Sort the DataFrame based on video_name and start_time in ascending order
df.sort_values(['video_name', 'start_time'], inplace=True)

df['transcription'] = df['transcription'].astype(str)
# Group the rows by video_name and concatenate the transcriptions
df_combined = df.groupby('video_name').agg({'transcription': ' '.join, 'label': 'first'}).reset_index()

df_combined.to_csv('../output/acoustic/combined_transcriptions.csv')


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score
import torch

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.argmax(torch.tensor(self.labels[idx]))
        return item

    def __len__(self):
        return len(self.labels)
    
# Load the combined DataFrame from the CSV file
df_combined = pd.read_csv('combined_transcriptions.csv')
df_combined = df_combined[['transcription', 'label']]
# Split the data into training, validation, and test sets
train_df, test_df = train_test_split(df_combined, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Define the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the input text and encode labels
train_encodings = tokenizer(list(train_df['transcription']), truncation=True, padding=True)
val_encodings = tokenizer(list(val_df['transcription']), truncation=True, padding=True)
test_encodings = tokenizer(list(test_df['transcription']), truncation=True, padding=True)

train_labels = list(train_df['label'])
val_labels = list(val_df['label'])
test_labels = list(test_df['label'])

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)
# Define the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define the training arguments for the Trainer
training_args = TrainingArguments(
    output_dir='../output/results',
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=0,
    weight_decay=0.01,
    logging_dir='./output/logs',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model on the test set
predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(axis=1)
accuracy = (pred_labels == test_labels).mean()

print("Test Accuracy:", accuracy)